In [ ]:
#----------------------------------------------------DATA MANIPULATION-------------------------------------------------------------------
import numpy as np
import pandas as pd
import talib
from scipy import stats

#Read raw data from file
stock_data = pd.read_csv("Data/sp500_stocks.csv")
df = pd.DataFrame(stock_data)
#Remove empty rows and unwanted columns
df = df.dropna(subset=["Close"])
df = df.drop(columns=["Adj Close", "Open", "Volume"])
#Convert date column to datetime
df["Date"] = pd.to_datetime(df["Date"])
#Sort the dataframe by symbol and then by date
df = df.sort_values(by=['Symbol', 'Date'])

#Find the number of entries for each symbol, calculate the mode, then reduce the data to this subset
#This means all symbols in this data have the same dates, and is the maximum such dataset
symbol_length = df.groupby(['Symbol']).count()
mode_samples = stats.mode(symbol_length["Date"])
symbols = list(symbol_length[symbol_length["Date"]==mode_samples[0][0]].index)
df = df.loc[df["Symbol"].isin(symbols)]

#Get a list of symbol data and size of all symbols
symbol_data = list(dict(tuple(df.groupby(['Symbol']))).values())
no_of_stocks = len(symbols)

#Loop through each day and calculate six indicators for each stock
IND_NUMBER = 6
for i in range(0, no_of_stocks):
    current = symbol_data[i]
    current["RSI"] = talib.RSI(current["Close"])
    current["MACD"] = talib.MACD(current["Close"])[0]
    current["CCI"] = talib.CCI(current["High"], current["Low"], current["Close"])
    current["ADX"] = talib.ADX(current["High"], current["Low"], current["Close"])
    current["NATR"] = talib.NATR(current["High"], current["Low"],current["Close"])
    current["ROCP"] = talib.ROCP(current["Close"], timeperiod=3)

#Add the indicators to the dataframe
df_formatted = pd.concat(symbol_data)

In [ ]:
#------------------------------------------------INDICATOR PLOT---------------------------------------------------------------

import matplotlib.pyplot as plt

chosen_symbol = "ALK"
chosen_date = "04-04-2017"
chosen_scale = 100

#Get subset of data under the timeframe and symbol name
df_restricted = df_formatted[(df_formatted["Symbol"]==chosen_symbol)&(df_formatted["Date"]>chosen_date)]

#Plot the figure
fig = plt.figure(figsize=(15,8))
plt.plot(df_restricted["Date"],df_restricted["ROCP"], label="Trend")
plt.plot(df_restricted["Date"],df_restricted["Close"]/chosen_scale, label="Close")
plt.legend(loc="upper left")
plt.title("Volatility")

plt.savefig("results/Volatility.png")

In [ ]:
#------------------------------------------------MORE DATA MANIPULATION--------------------------------------------------------------

#Function that returns subset of data within given timeframe

def split_data(df,start,end):
    data = df[(df["Date"]>= start) & (df["Date"]< end)]
    #Reset index to incremental numbers
    data.index = data["Date"].factorize()[0]
    return data
    
#Find the first and last date in the dataset
first_date = df_formatted["Date"].min()
last_date = df_formatted["Date"].max()

#Set the start of training and trading
training_start = "2010-04-04"
trading_start = "2017-04-04"

#Set the number of stocks we want to invest in
STOCK_DIM = 40

#Get data from first N years and get list of all dates
year_offset = 5
df_current_year = split_data(df_formatted, first_date, first_date + pd.offsets.DateOffset(years=year_offset))
all_dates = list(df_current_year["Date"].unique())

#Get entries at first and last date, then compute percentage change between them
df_first_last = df_current_year[(df_current_year["Date"]==all_dates[0]) | (df_current_year["Date"]==all_dates[-1])][{"Symbol","Close"}]
df_fl_returns = df_first_last.groupby("Symbol").pct_change().dropna()
#Add symbol row, then sort values by close (this holds the percentage change)
#Stocks are therefore sorted by their rate of return in the first 3 years
df_fl_returns["Symbol"]=df_current_year["Symbol"].unique()
df_fl_returns = df_fl_returns.sort_values(by="Close", ascending=False)

#Choose the stocks we trade by taking a random subset of the top 100 performing stocks
#stock_choice = list(df_fl_returns["Symbol"][0:STOCK_DIM])
performing_stocks = list(df_fl_returns["Symbol"])[0:100]
stock_choice = [performing_stocks[i] for i in np.random.choice(range(0,len(performing_stocks)),STOCK_DIM,replace=False)]
print(stock_choice)

#Create final dataset by removing all unwanted stocks
#(Note that df_final is sorted by symbol (alphabetically) and then by date (chronologically))
df_final = df_formatted.loc[df_formatted["Symbol"].isin(stock_choice)]

In [ ]:
#--------------------------------------------------RUN STRATEGY-----------------------------------------------------------------

from stable_baselines.common.vec_env import DummyVecEnv

#Function that runs the training and then trading strategy
#Takes in the dataset, the training/trading start times, the number of simulations to run during training, and any hyperparameters

def run_strategy(df, start_times, no_sims,hyperparams):
    print("--------------Starting Strategy------------------")
    
    #Set training start/end times and split accordingly
    start_date = start_times[0]
    end_date = start_times[1]
    training_data = split_data(df,start_date, end_date)

    #Setup the training environment using Stable Baselines
    training_env_DDPG = DummyVecEnv([lambda: StockEnv(df=training_data, env_type="train", VOLATILITY_THRESHOLD=hyperparams[0], HOLD_THRESHOLD=hyperparams[1])])
    training_env_A2C = DummyVecEnv([lambda: StockEnv(df=training_data, env_type="train", VOLATILITY_THRESHOLD=hyperparams[0], HOLD_THRESHOLD=hyperparams[1])])


    #Calculate the number of times we need to run the training steps
    total_days = no_sims * (len(training_data["Date"].unique())) 
    
    #Train the model
    trained_model_DDPG = train_model(training_env_DDPG, total_days, "DDPG")
    trained_model_A2C = train_model(training_env_A2C, total_days, "A2C")
    #Trade the model
    trade_DDPG = trade_model(df=df, model=trained_model_DDPG, start_times = start_times, hyperparams=hyperparams)
    trade_A2C = trade_model(df=df, model=trained_model_A2C, start_times = start_times, hyperparams=hyperparams)
    
    return [trade_DDPG,trade_A2C]

In [ ]:
#--------------------------------------------------------TRAINING ENVIRONMENT--------------------------------------------------------------
import gym
from gym import spaces
import matplotlib.pyplot as plt

#Set the starting wealth of the treader to 1 million
STARTING_BALANCE = 1000000

#Risk free rate - defining it here speeds up the running
rfr = 0.03
daily_rfr = (1+rfr)**(1/365) - 1
annualised_days = np.sqrt(252)

#This factor adjusts the action space to a suitable range, since some models require an action space normalised around [-1,1]
MULT_FACTOR = 75

#Class for our stock environment, which holds all the required classes for a custom Stable Baselines environment
class StockEnv(gym.Env):
    
    metadata = {'render.modes': ['human']}
    
    #Initialise the environment and set variables to starting vallues
    def __init__(self,df,day=0, env_type="train", VOLATILITY_THRESHOLD = -0.09, HOLD_THRESHOLD = 15):
        self.VOLATILITY_THRESHOLD = VOLATILITY_THRESHOLD
        self.HOLD_THRESHOLD = HOLD_THRESHOLD
        
        self.iteration = 0
        self.day = day
        self.df = df
        self.env_type = env_type

        #Action space generates a vector of size STOCK_DIM of random values in [-1,1]
        self.action_space = spaces.Box(low=-1,high=1, shape = (STOCK_DIM,))    
        
        #Observation space contains all required data for our model to base its predictions off of
        #Equals [Starting Wealth] + [Stock Values] + [Owned Shares in Each Stock] + [All Indicator Data]
        self.observation_space = spaces.Box(low=0, high=np.inf, shape = (STOCK_DIM*(2 + IND_NUMBER)+1,))

        #Get all data for the current day
        self.data = self.df.loc[self.day,:]

        #Set the state accordingly for observations
        self.state = [STARTING_BALANCE] + self.data["Close"].tolist() + [0]*STOCK_DIM + \
                        self.data["RSI"].values.tolist() + \
                        self.data["MACD"].values.tolist() + \
                        self.data["CCI"].values.tolist() + \
                        self.data["ADX"].values.tolist() + \
                        self.data["NATR"].values.tolist() + \
                        self.data["ROCP"].values.tolist()
       
        self.daily_total = [STARTING_BALANCE]
    

    #Function to sell a specified number of shares in a stock
    def sell_stock(self, index, action):
        #Check there are shares to actually sell
        if self.state[index+STOCK_DIM+1]>0:
            #If the stock is too volatile, sell off all shares
            if (self.state[index+STOCK_DIM*7+1]<self.VOLATILITY_THRESHOLD):
                action = 10000 * action

            #Calculate how many shares we can sell (caps out at the number we currently own)
            action_cost = min(abs(action), self.state[index+STOCK_DIM+1])

            self.state[0] += self.state[index+1]*action_cost
            self.state[index+STOCK_DIM+1] -= action_cost
        else:
            #Do nothing if there are no shares to sell
            pass
    
    #Function to hold a specified number of shares in a stock
    def hold_stock(self, index, action):
        #If the stock is too volatile, sell off all shares
        if (self.state[index+STOCK_DIM*7+1]<self.VOLATILITY_THRESHOLD):
            self.sell_stock(index,-abs(10000*action))
        else:
            #Do nothing otherwise, ie hold
            pass

    #Function to buy a specified number of shares in a stock
    def buy_stock(self, index, action):
        #If the stock is too volatile, sell off all shares
        if (self.state[index+STOCK_DIM*7+1]<self.VOLATILITY_THRESHOLD):
            self.sell_stock(index,-10000*action)
        else:
            #Find number of shares we can afford to buy
            available_amount = self.state[0] // self.state[index+1]

            #Calculate how many shares we can  buy
            action_cost = min(available_amount, action)

            self.state[0] -= self.state[index+1]*action_cost
            self.state[index+STOCK_DIM+1] += action_cost
        
    #Function which runs trading for one day
    def step(self,actions):
        #Calculates if we have reached the end of the data 
        final_day = self.day >= len(self.df.index.unique())-1
        #Calculate starting wealth: Bank Balance + [stock_amount*stock_price] for each stock
        pre_trade_wealth = self.state[0]+ sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
        #Initialise post_trade_wealth
        post_trade_wealth = 0

        if not final_day:
            #Convert action space into more usable values
            actions = MULT_FACTOR * actions 
            
            #Generate lists of all stocks that we are selling, buying, and holding
            action_index = np.argsort(actions)
            sell_index = action_index[:np.where(actions < -self.HOLD_THRESHOLD)[0].shape[0]]
            buy_index = action_index[::-1][:np.where(actions > self.HOLD_THRESHOLD)[0].shape[0]]
            hold_index = action_index[np.where(actions>=-self.HOLD_THRESHOLD)[0].shape[0]:np.where(actions <= self.HOLD_THRESHOLD)[0].shape[0]]
        
            #Check for market volatility
            thresh_count=0
            for index in action_index:
                if (self.state[index+STOCK_DIM*7+1]<self.VOLATILITY_THRESHOLD):
                    thresh_count += 1

            #Sell, buy and hold the stocks for the day
            #We add/substract hold theshold to renormalise to zero (since otherwise values close to zero would only ever appear for holds)
            
            if (thresh_count <STOCK_DIM / 3):
                for index in sell_index:
                    self.sell_stock(index, actions[index]+self.HOLD_THRESHOLD)
                for index in buy_index:
                    self.buy_stock(index, actions[index]-self.HOLD_THRESHOLD)
                for index in hold_index:
                    self.hold_stock(index,actions[index])
            else:
                for index in action_index:
                    self.sell_stock(index, actions[index]*10000)
                # if (self.env_type=="trade"):
                #     print("SELLLLLL")
                #     print(self.state[41:81])
            
            #Increment the day and dataset
            self.day += 1
            self.data = self.df.loc[self.day,:] #This is ordered alphabetically by symbol
                 
            #Set the current set to the next day of trading
            self.state = [self.state[0]] + self.data["Close"].values.tolist()+list(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]) + \
                self.data["RSI"].values.tolist() + self.data["MACD"].values.tolist() + self.data["CCI"].values.tolist() + \
                self.data["ADX"].values.tolist() + self.data["NATR"].values.tolist() + self.data["ROCP"].values.tolist()
            
            #Calculate the post-trade wealth by summing the bank balance and the updated stock number/price
            post_trade_wealth = self.state[0] + sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))

            #Update daily wealth
            self.daily_total.append(post_trade_wealth)

        else:
            #Increment the iteration number
            self.iteration += 1

            #Calculate final wealth
            post_trade_wealth = self.state[0] + sum(np.array(self.state[1:(STOCK_DIM+1)]) * np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
            self.daily_total.append(post_trade_wealth)

            #Plot graph
            plt.plot(self.daily_total,'r')
            plt.savefig('results/account_value_{}_{}.png'.format(self.env_type,str(self.iteration)))
            plt.close()
            
            #Calculate sharpe ratio
            df_total_value = pd.DataFrame(self.daily_total)
            df_total_value.columns = ["Wealth"]
            df_total_value["Return"]=df_total_value.pct_change(1)
            sharpe = annualised_days*(df_total_value["Return"].mean()-daily_rfr)/df_total_value["Return"].std()

            #Calculate overall return
            final_return = (self.daily_total[-1]/STARTING_BALANCE -1)

            print("Wealth:",round(post_trade_wealth,0))
            print("Return:", round(final_return*100,3),"%")
            print("Annual Return", round(final_return * 252 / len(df_total_value["Return"])*100,3),"%")
            print("Sharpe: ",round(sharpe,3))
            print("---------------------------")

        #Calculate the change in wealth and use that as our reward value
        daily_reward = post_trade_wealth - pre_trade_wealth
                
        return self.state, daily_reward, final_day, {}    
        
    #Reset the environment to starting values
    def reset(self):
        self.daily_total = [STARTING_BALANCE]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        self.state = [STARTING_BALANCE] + self.data["Close"].tolist() + [0]*STOCK_DIM + \
                    self.data["RSI"].values.tolist() + \
                    self.data["MACD"].values.tolist() + \
                    self.data["CCI"].values.tolist() + \
                    self.data["ADX"].values.tolist()  + \
                    self.data["NATR"].values.tolist() + \
                    self.data["ROCP"].values.tolist()

        return self.state
    
    def return_data(self):
        return self.daily_total

    #Other functions required for Stable Baselines
    def render(self, mode='human', close=False):
        return self.state

    def close(self):
        pass
        

In [ ]:
#----------------------------------------------------MODEL TRAINING------------------------------------------------------------

import time
import gym
from stable_baselines import A2C,DDPG
from stable_baselines.common.noise import OrnsteinUhlenbeckActionNoise
    
#This function trains the model by running DDPG on the given dataset, then returns the model

def train_model(training_env, total_days, algorithm_type):
    
    #Measure how long the training takes
    start = time.time()
    
    if (algorithm_type=="DDPG"):
        #Define variables for DDPG
        n_actions = training_env.action_space.shape[0]
        param_noise = None
        action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions),sigma=float(0.5) * np.ones(n_actions))
        #Create the model using Stable Baselines
        model = DDPG('MlpPolicy', training_env, verbose = 0, param_noise = param_noise, action_noise = action_noise)
    else:
        model = A2C('MlpPolicy', training_env, verbose=0)

    #Train the model
    model.learn(total_timesteps=total_days)
    
    end = time.time()
    print("Training Time:", round((end - start) / 60,2), "Minutes")

    return model

In [ ]:
#This function starts trading using the model we have just trained

def trade_model(df, model, start_times, hyperparams):
    #Remove early dates from the dataset
    trading_data = split_data(df, start_times[1], last_date)
    #Setup the trading environment
    trading_env = DummyVecEnv([lambda: StockEnv(df=trading_data, env_type="trade",VOLATILITY_THRESHOLD=hyperparams[0], HOLD_THRESHOLD=hyperparams[1])])
    #Reset the trading environment
    trading_obs = trading_env.reset()

    #Loop through each date in the dataset and day-trade based off the model
    
    
    for i in range(len(trading_data.index.unique())):
        #Predict the best action to take based off the data
        action = model.predict(trading_obs)[0]
        #Make the action and update the environment
        trading_obs = trading_env.step(action)[0]

        #Get daily wealth throughout trading
        if i == len(trading_data.index.unique()) - 2:
            all_wealth_data = trading_env.env_method("return_data")
    
    return all_wealth_data[0]


In [ ]:
#--------------------------------------------------RUN STRATEGY--------------------------------------------------------------

hyperparams = [-0.09, 15]
no_sims = 25

#Run the RL strategy
daily_wealth = run_strategy(df = df_final, start_times=[training_start, trading_start], no_sims = no_sims, hyperparams=hyperparams)

#------------------------------HYPERPARAMETER OPTIMISATION---------------------------------#
#Loop through different parameter settings
# for vol_thresh in [-0.15,-0.13,-0.11,-0.09,-0.07,-0.05,-0.03, 0]:
#     for hold_thresh in [10,15,20]:
#         print(vol_thresh)
#         print(hold_thresh)
#         daily_wealth = run_strategy(df = df_final, start_times=[training_start, trading_start], no_sims = no_sims, hyperparams = [vol_thresh, hold_thresh])

In [ ]:
#--------------------------------------------------S&P DATA-------------------------------------------------------------

#Load in all S&P index data for all stocks past the treadng date
df_index = pd.DataFrame(pd.read_csv("Data/sp500_index.csv"))
df_index = df_index[df_index["Date"]>= trading_start]
#Normalise so that the starting value is at the starting wealth
normalisation = STARTING_BALANCE / df_index["S&P500"].iloc[0]
df_index["normalised"] = df_index["S&P500"]  * normalisation

y1 = list(df_index["normalised"])

In [ ]:
#----------------------------------------------OPTIMAL PORTFOLIO---------------------------------------------------------

#Base portfolio off of training data times, splitting as such
df_train_port = split_data(df_final,training_start,trading_start)
df_trade_port = split_data(df_final,trading_start,last_date)

#Manipulate the trading data into a new dataframe with columns as symbols, and rows as dates
df_by_symbol = pd.DataFrame(df_train_port["Date"].sort_values().drop_duplicates(), columns=["Date"])
for symbol in stock_choice:
    df_by_symbol[symbol]=df_train_port[df_train_port["Symbol"]==symbol]["Close"].sort_values()
    
#Remove dates from the dataframe
_df_by_symbol=df_by_symbol.drop(columns=["Date"])
#Calculate logarithmic covariance and correlation matrices (log makes it a lot more efficient)
cov_matrix = _df_by_symbol.pct_change().apply(lambda x: np.log(1+x)).cov()
corr_matrix = _df_by_symbol.pct_change().apply(lambda x: np.log(1+x)).corr()

#Consider whole training period
return_period = [training_start, trading_start]

#Calculate return over this period for each symbol and order by return
df_current_year = split_data(df_by_symbol, return_period[0], return_period[1]).drop(columns=["Date"])
change = df_current_year.iloc[[0,-1]].pct_change().iloc[1]

df_returns = pd.DataFrame({"Symbol":stock_choice, "Return":change})

weights = []
returns = []
vols = []

#Generate N different portfolio weightings (these sum up to 1)
portfolio_num = 1000
for i in range(1,portfolio_num):
    weighting = np.random.random(len(stock_choice))
    weighting = weighting/np.sum(weighting)
    weights.append(weighting)
    
    #Calculate the total return of the portfolio
    total_return = np.dot(weighting, list(df_returns["Return"]))
    returns.append(total_return)
    #Calculate the total variance and annualised variance
    total_var = cov_matrix.mul(weighting,axis=0).mul(weighting, axis=1).sum().sum()
    total_ann = np.sqrt(252 * total_var)
    vols.append(total_ann)

#Store risk and return values for each porfolio    
retvol = {"Returns":returns, "Vol":vols}
for counter, symbol in enumerate(_df_by_symbol.columns):
    retvol[symbol + " weight"] = [w[counter] for w in weights]
#Plot all portfolios
df_port = pd.DataFrame(retvol)
df_port.plot.scatter(x="Vol",y="Returns",s=1)

#Find portfolio with best sharpe ratio - rfr might be incorrect here...
optimal_port = df_port.iloc[((df_port['Returns']-rfr)/df_port['Vol']).idxmax()]

In [ ]:
#--------------------------------------TRADE OPTIMAL PORTFOLIO--------------------------------------------------

import matplotlib.pyplot as plt
#Find portfolio based on this optimal weighting
portfolio = list(optimal_port)
#Take 2nd index onwards, returns [0.009, 0.03,0.01,...] unordered CPB, MSI, LNT, etc...
portfolio = portfolio[2:]
#Map symbols to the portfolio weighting
stock_mapping = dict(zip(stock_choice, portfolio)) 
#Get starting date
starting_date = df_trade_port["Date"].sort_values().iloc[0]
#Get amount of money we are investing in each stock (stocks are not alphabetically ordered)
investment = [z * STARTING_BALANCE for z in portfolio]
#Get prices of each stock for the current date (stocks ARE alphabetically ordered)
all_stock_prices = df_trade_port[df_trade_port["Date"]==starting_date][{"Symbol","Close"}]
#Reduce down to our stock portfolio
chosen_stock_prices = all_stock_prices[all_stock_prices["Symbol"].isin(stock_choice)]
#Reorder our portfolio so that stocks are alphabetical
ordered_portfolio = [stock_mapping[symbol] for symbol in chosen_stock_prices["Symbol"]]
#Calculate the amoutn of money were are investing in each stock (stocks are alphabetically ordered)
ordered_investment = [z * STARTING_BALANCE for z in ordered_portfolio]
#Find how many shares we are purchasing in each stock (allowing fractional shares)
no_of_stocks = [i/j for i,j in zip(ordered_investment, chosen_stock_prices["Close"])]

#Get all dates
x2 = list(df_trade_port["Date"].unique())
y2= [None] * len(x2)

#Calculate the portfolio price for each day
for i in range(0,len(x2)):
    #Get current date
    date_curr = x2[i]
    #Find the stock prices for the day
    stock_prices = df_trade_port[df_trade_port["Date"]==date_curr]
    #Calculate the current value of each of our stock investments
    stock_values = [i*j for i,j in zip(no_of_stocks, list(stock_prices["Close"]))]
    #Get the total portfolio wealth
    y2[i] = np.sum(stock_values)


In [ ]:
#-----------------------------------------------------PLOTTING----------------------------------------------------------------

#Get all the dates during trading and reset the index
unique_dates = df_final["Date"].sort_values()
unique_dates = pd.DataFrame(unique_dates.drop_duplicates())
unique_dates = unique_dates.reset_index()
#Find the first starting date
starting_date = unique_dates[unique_dates["Date"]>= trading_start].index[0]

#Get the Reinforcement Learning portfolio
x = list(unique_dates["Date"])[starting_date:]
y_ddpg = daily_wealth[0]
y_a2c = daily_wealth[1] 

#Set up the figure
fig = plt.figure(figsize=(18,7))
plt.rcParams["xtick.labelsize"] = 14
plt.rcParams["ytick.labelsize"] = 14
#Plot the Reinforcement Learning Portfolio
plt.plot(x,y_ddpg, "-b", label="DDPG")
plt.plot(x,y_a2c,"-y", label="A2C")
#Plot the Optimal Portfolio
plt.plot(x,y1, "-r", label="S&P 500")
#Plot the S&P Index Portfolio
plt.plot(x2,y2, "-g", label="Portfolio Optimisation Theory")

#Display and save the plot
plt.legend(loc="upper left", fontsize = 14)
plt.title("Portfolio Value over Time", fontsize = 20)
plt.xlabel("Time", fontsize = 18)
plt.ylabel("Total Wealth", fontsize = 18)
plt.savefig("results/Trade Results.png", dpi=300)
plt.show()
